# Import libraries

In [ ]:
import sigma
from sigma.utils import normalisation as norm 
from sigma.utils.load import IMAGEDataset
from sigma.src.utils import same_seeds
from sigma.src.dim_reduction import Experiment
from sigma.models.autoencoder import AutoEncoder
from sigma.src.segmentation import PixelSegmenter
from sigma.gui import gui

# Load files

Load the image files from `chemical_maps_dir` and the intensity map file using the path `intensity_map_path`, creating an object of `IMAGEDataset`.

In [ ]:
chemical_maps_dir='/'
intensity_map_path='.'
im = IMAGEDataset(chemical_maps_dir,intensity_map_path)

# Dataset preprocessing

In [ ]:
gui.view_im_dataset(im)

## Normalisation

In [ ]:
im.rebin_signal(size=(4,4))

In [ ]:
# Normalise the dataset using the (optional) sequential three methods.
im.normalisation([norm.neighbour_averaging,
                  norm.zscore])
                  # norm.softmax])

Use `gui.view_pixel_distributions` to view the intensity distributions after each sequential normalisation process.

In [ ]:
gui.view_pixel_distributions(dataset=im, 
                             norm_list=[norm.neighbour_averaging,
                                        norm.zscore,
                                        norm.softmax], 
                             cmap='inferno')

## (Optional) Assign RGB to elemental peaks

In [ ]:
gui.view_rgb(im)

## Check elemental distribution after normalisation

In [ ]:
print('After normalisation:')
gui.view_intensity_maps(spectra=im.normalised_elemental_data, element_list=im.feature_list)

# 2. Dimensionality reduction

## 2.2 Method 2: UMAP

In [ ]:
from umap import UMAP

# Parameter tuning can be found https://umap-learn.readthedocs.io/en/latest/parameters.html
data = im.normalised_elemental_data.reshape(-1,len(im.feature_list))
umap = UMAP(
        n_neighbors=10,
        min_dist=0.1,
        n_components=2,
        metric='euclidean'
    )
latent = umap.fit_transform(data)

# 3. Pixel segmentation: 

## 3.2 Method 2: HDBSCAN clustering

In [ ]:
# hyperparameter tuning can be found https://scikit-learn.org/stable/auto_examples/cluster/plot_hdbscan.html#hyperparameter-robustness
ps = PixelSegmenter(latent=latent, 
                    dataset=im,
                    method="HDBSCAN",
                    method_args=dict(min_cluster_size=50, min_samples=35,
                                     max_cluster_size=int(len(latent)/10),
                                     cluster_selection_epsilon=5e-2) )

## Checking latent space

In [ ]:
# Plot latent sapce (2-dimensional) with corresponding Gaussian models
gui.view_latent_space(ps=ps, color=True)

In [ ]:
# visualise the latent space
gui.check_latent_space(ps=ps,ratio_to_be_shown=1.0, show_map=True)

In [ ]:
# check the density of latent space
gui.plot_latent_density(ps=ps, bins=50)

## Checking each clusters

In [ ]:
gui.show_cluster_distribution(ps=ps)

## Checking cluster map

In [ ]:
# Plot phase map using the corresponding GM model
gui.view_phase_map(ps=ps)